In [1]:
# load module and pre generate data
from graph_tree_link_module import *

import numpy as np

import pickle

# Open the file in binary read mode and load the object
with open("graph_tree.pkl", "rb") as file:
    gt = pickle.load(file)


In [2]:
sliders_data = [{"id": gt.attr_name[i], "min": gt.X_maxmin[1][i], "max": gt.X_maxmin[0][i], "value": gt.nodes[0].center[i], "type": "f"} for i in range(len(gt.attr_name))]
# print(sliders_data)

recom_data = [{"i": i, "id": gt.attr_name[i], "max_val": gt.recom[i][0].center[i], "min_val": gt.recom[i][1].center[i]} for i in range(len(gt.attr_name))]
# print(recom_data)

tree_data = {
        "name": "Root",
        "children": [
            {
                "name": "Child 1",
                "children": [
                    {"name": "Grandchild 1"},
                    {"name": "Grandchild 2"}
                ]
            },
            {
                "name": "Child 2",
                "children": [
                    {"name": "Grandchild 3"},
                    {"name": "Grandchild 4"}
                ]
            }
        ]
    }

In [3]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

# slider ####################################################################
@app.route('/sliders')
def sliders():
    return jsonify(sliders_data)

@app.route('/handle_slider', methods=['POST'])
def submit_slider_values():
    data = request.json
    print('Received slider values:', data)
    query = np.array([float(val) for val in data.values()])
    cur = gt.search(query)
    print(cur.id)
    print(gt.status)
    global graph_data
    graph_data = get_suround_node_data(gt.status)
    # Process the data as needed
    return jsonify({'status': 'slider respon success', 'received': data})
# slider end ################################################################

# recom ####################################################################
@app.route('/recom')
def recom():
    return jsonify(recom_data)

@app.route('/handle_recom', methods=['POST'])
def handle_recom():
    data = request.json
    button_id = int(data.get('buttonId'))
    val = int(data.get('val'))
    print(button_id, val)
    moved_node = gt.recom[button_id][val]
    gt.get_suround_cluster(moved_node)
    global graph_data
    graph_data = get_suround_node_data(gt.status)
    # Handle the button click based on the button_id
    return jsonify({'status': 'recom respon success', 'buttonId': button_id, 'val': val, "data": graph_data})
# recom end ####################################################################

@app.route('/graph-data')
def graph():
    return jsonify(graph_data)

@app.route('/tree-data')
def tree():
    return jsonify(tree_data)

if __name__ == '__main__':
    app.run(port=8123)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8123
Press CTRL+C to quit
127.0.0.1 - - [29/Sep/2024 11:45:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2024 11:45:36] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Sep/2024 11:45:36] "GET /static/scripts.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Sep/2024 11:45:36] "GET /static/graph_tree.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2024 11:45:36] "GET /static/curve_axis.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Sep/2024 11:45:36] "GET /sliders HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2024 11:45:36] "GET /recom HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2024 11:45:36] "GET /tree-data HTTP/1.1" 200 -
[2024-09-29 11:45:36,670] ERROR in app: Exception on /graph-data [GET]
Traceback (most recent call last):
  File "C:\Users\lst74\AppData\Roaming\Python\Python312\site-packages\flask\app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lst74\AppData\Roaming\Python\Python312\site-packa